In [ ]:
import numpy as np
import gsd
import gsd.experimental as gsde

import matplotlib.pyplot as plt
import jax
import jax.numpy as jnp
import tensorflow_probability.substrates.jax as tfp
tfd = tfp.distributions
tfb = tfp.bijectors
import requests
from functools import partial
from gsd.fit import make_logits, allowed_region, GSDParams
from jax.typing import ArrayLike
from jax import Array
from jax.flatten_util import ravel_pytree


Lets use one experiment form sureal library

In [ ]:
url = 'https://raw.githubusercontent.com/Netflix/sureal/master/test/resource/NFLX_dataset_public_raw.py'  # Replace with your URL
dataset={}
try:
    response = requests.get(url)
    if response.status_code == 200:
        content = response.text
        exec(content,{},dataset)
    else:
        print(f"Failed to retrieve the file. Status code: {response.status_code}")
except requests.RequestException as e:
    print(f"Error fetching the file: {e}")


In [ ]:
o=np.asarray([v['os'] for v in dataset['dis_videos']])
print(o.shape)
counts=jax.vmap(gsd.sufficient_statistic)(o)

In [ ]:
hdtv=False
hdtv=True

In [ ]:
import pandas as pd
if hdtv:
    hdtv=pd.read_csv("/Users/krzysiek/Documents/lts_analysis_soft/pyits/log/hdtv_data.csv")
    exp1 = hdtv[hdtv.Experiment==1]
    exp1
    n_sub=24
    n_pvs=168
    o = np.zeros((n_pvs, n_sub))
    o = pd.pivot(exp1, columns='Tester_id', values='Score', index='PVS_id').to_numpy()
    o
    counts=jax.vmap(gsd.sufficient_statistic)(o)
    
    

In [ ]:
@jax.jit
def gsdfit(x:Array):
    params,opt_state = gsde.fit_mle(data=x, max_iterations=200)
    return params

Fit model for a single PVS

In [ ]:
gsdfit(counts[0])

And compare the fit to the one estimated without a gradient:


In [ ]:
theta0 = GSDParams(psi=2.0, rho=0.9)
x0, unravel_fn = ravel_pytree(theta0)

def nll(x: ArrayLike, data: Array) -> Array:
    logits = make_logits(unravel_fn(x))
    tv = allowed_region(logits,data.sum())
    ret = jnp.where(tv,-jnp.dot(logits, data), jnp.inf)
    
    return ret
@jax.jit
def tfpfit(data:Array):
    initial_simplex = np.asarray(
    [
        [4.9, 0.1],
        [1.1, 0.9],
        [4.9, 0.9],
    ]
)
    results = tfp.optimizer.nelder_mead_minimize(
        partial(nll, data=data),
        initial_simplex = jnp.asarray(initial_simplex)
    )
    return unravel_fn(results.position)


In [ ]:
[gsdfit(counts[0]), tfpfit(counts[0])]

Let's estimate para,eter for all the PVSs. 
For this we are going to use `jax.lax.map`.
 _Note, that `vmap` is nor best here as each estimatio contain control flow instructions._
  
 

In [ ]:
from gsd.experimental.fit import GridEstimator


fits = jax.lax.map(gsdfit, counts)
grid = GridEstimator.make(GSDParams(512,512))

In [ ]:
n=40
n=3
print(counts[n])
print(jax.tree_util.tree_map(lambda x:x[n],fits))

print(tfpfit(counts[n]))
print(grid(counts[n]))
print(gsde.fit_mle_grid(counts[n],num=GSDParams(128,64),constrain_by_pmax=False))

# PP-plot

In [ ]:
key = jax.random.key(42)
keys = jax.random.split(key, counts.shape[0])
from gsd.experimental.fit import GridEstimator



pvals=np.stack([gsde.pp_plot_data(c, estimator=grid,key=key, n_bootstrap_samples=99) for c,key in zip(counts, keys)])
pvals


In [ ]:
from scipy.stats import norm, uniform

def pp_plot(pvalues:np.ndarray, thresh_pvalue=0.2):
    try:
        import matplotlib.pyplot as plt
    except:  # noqa: E722
        raise NotImplementedError()

    n_pvs = len(pvalues)
    ref_p_values = np.linspace(start=0.001, stop=thresh_pvalue, num=100)
    significance_line = ref_p_values + norm.ppf(0.95) * np.sqrt(
        ref_p_values * (1 - ref_p_values) / n_pvs)

    def count_pvs_fraction(p_value, p_value_per_pvs):
        return jnp.sum(p_value_per_pvs <= p_value) / len(p_value_per_pvs)
    


    pvs_fraction_gsd = np.asarray(jax.vmap(count_pvs_fraction, in_axes=(0,None))(pvalues,pvalues))

    plt.scatter(pvalues, pvs_fraction_gsd, label="GSD")

    plt.xlabel("theoretical uniform cdf")
    plt.ylabel("ecdf of $p$-values")
    plt.plot(ref_p_values, significance_line, "-k")
    plt.xlim([0, thresh_pvalue])
    plt.ylim([0, thresh_pvalue + 0.1])
    plt.minorticks_on()
    plt.show()

pp_plot(pvals)